# Tabular exploration

In [31]:
import db
import parser
import pandas as pd
from datetime import datetime, timedelta

In [32]:
dev = db.dev()
begin = datetime.today() + timedelta(days=-30)

In [33]:
logEntries = dev.AppUpdateLog.find({"createdAt": {"$gt": begin}})  #.limit(2)

In [34]:
parser.pretty(logEntries)

<IPython.core.display.JSON object>

### A database aggregation has a pipeline with multiple stages
In the pibeline below I have the following stages:
- Stage 1: Fetch the latest entries, using `$match`
- Stage 2: Group by two fields and counts the entries, using `$group`
- Stage 3: Flatten the result into a tabular structure, using `$project`
- Stage 4: Sort the data, using `$sort`

In [35]:
statuses = list(dev.AppUpdateLog.aggregate([
    {"$match": { "createdAt": {"$gt": begin} }},
    {"$group": { "_id": {"status": "$status", "os": "$clientType"}, "count": { "$sum": 1 }}},
    {"$project": {"status": '$_id.status', "os": '$_id.os', "count": "$count"}},
    {"$sort": {"count": -1, "os": 1}}
]))

### Using DataFrames, data can be presented with formatting

In [36]:
pd.DataFrame(statuses)

,_id,status,os,count
0,"{'status': 'rejected', 'os': 'ios'}",rejected,ios,3
1,"{'status': 'succeeded', 'os': 'web'}",succeeded,web,3
2,"{'status': 'succeeded', 'os': 'ios'}",succeeded,ios,1


In [37]:
statuses = list(dev.AppUpdateLog.aggregate([
    {"$match": { "createdAt": {"$gt": begin} }},
    {"$unwind": { "path": "$log", "includeArrayIndex": "logIndex"}},
    {"$project": {"timestamp": "$log.timestamp", 
                  "user": "$userId", 
                  "logIndex": 1, 
                  "os": "$clientType", 
                  "version": "$log.clientVersion", 
                  "status": "$log.status",
                  "finalStatus": "$status"}},
    {"$sort": {"timestamp": 1}}
]))

pd.DataFrame(statuses).drop(["_id"], axis=1)

,logIndex,timestamp,user,os,version,status,finalStatus
0,0,2022-09-22 07:43:56.694,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
1,1,2022-09-22 07:43:56.981,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
2,2,2022-09-22 07:44:25.865,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
3,3,2022-09-22 07:44:26.115,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
4,4,2022-09-22 07:48:24.467,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
5,5,2022-09-22 07:48:24.725,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
6,6,2022-09-22 07:52:47.456,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
7,7,2022-09-22 07:52:47.715,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
8,8,2022-09-22 07:52:50.741,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
9,9,2022-09-22 07:52:51.016,63118ee37da4395a758f5c09,web,2.3.3,rejected,succeeded
